In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.layers import *
from models.baseline_model import ConvLSTM_Model
from preprocess.data_reader import data_reader
from preprocess.cvt2tfrecord import png_cvter

from IPython.core.debugger import set_trace
from pathlib import Path
from tqdm import tqdm_notebook

# tfrecord

## Create tfrecord

In [46]:
PATH = Path('../data')
path_trn_1 = PATH/'SRAD2018_TRAIN_001'
path_tfrecord = PATH/'train_001.tfrecords'

In [54]:
import os
os.listdir(str(path_trn_1/'RAD_386582464212543'))

['RAD_386582464212543_040.png',
 'RAD_386582464212543_010.png',
 'RAD_386582464212543_056.png',
 'RAD_386582464212543_020.png',
 'RAD_386582464212543_060.png',
 'RAD_386582464212543_000.png',
 'RAD_386582464212543_026.png',
 'RAD_386582464212543_017.png',
 'RAD_386582464212543_018.png',
 'RAD_386582464212543_003.png',
 'RAD_386582464212543_004.png',
 'RAD_386582464212543_036.png',
 'RAD_386582464212543_031.png',
 'RAD_386582464212543_014.png',
 'RAD_386582464212543_033.png',
 'RAD_386582464212543_039.png',
 'RAD_386582464212543_008.png',
 'RAD_386582464212543_043.png',
 'RAD_386582464212543_032.png',
 'RAD_386582464212543_038.png',
 'RAD_386582464212543_002.png',
 'RAD_386582464212543_046.png',
 'RAD_386582464212543_047.png',
 'RAD_386582464212543_052.png',
 'RAD_386582464212543_019.png',
 'RAD_386582464212543_053.png',
 'RAD_386582464212543_042.png',
 'RAD_386582464212543_044.png',
 'RAD_386582464212543_023.png',
 'RAD_386582464212543_001.png',
 'RAD_386582464212543_006.png',
 'RAD_38

In [47]:
cvt = png_cvter(file_name=str(path_tfrecord), data_dir = str(path_trn_1), 
                hight=501, width=501)

In [49]:
cvt.convert()

## Decode tfrecord

In [12]:
dataset = tf.data.TFRecordDataset(str(path_tfrecord))

In [ ]:
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_jpeg(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

# A vector of filenames.
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])

# `labels[i]` is the label for the image in `filenames[i].
labels = tf.constant([0, 37, ...])

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.map(_parse_function)

In [27]:
def parser_train(record):
    features = tf.parse_single_example(record, 
                features={
                    'data_raw': tf.FixedLenFeature([], tf.string),
                    'label_raw': tf.FixedLenFeature([], tf.string)
                })
    image = tf.image.decode_png(features['data_raw'], channels=1)
    image = tf.reshape(image, [31, 501, 501, 1])
    image = tf.cast(image, tf.float32)
    
    label = tf.image.decode_png(features['label_raw'], channels=1)
    label = tf.reshape(label, [30, 501, 501, 1])
    label = tf.cast(label, tf.float32)
    return image, label

In [29]:
batch_size = 1

filenames = tf.placeholder(tf.string, shape=[None])
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser_train).repeat().batch(batch_size)
    
iterator = dataset.make_initializable_iterator()

training_filenames = [str(path_tfrecord)]
with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={filenames: training_filenames})
    iterator

In [17]:
with open('../data/SRAD2018_TRAIN_001/RAD_206482464219531/RAD_206482464219531_060.png', 'rb') as f:
     png_bytes = f.read()
bytes = tf.placeholder(tf.string)
decode_png = tf.image.decode_image(bytes, channels=3)
session = tf.Session()
image = session.run(decode_png, feed_dict={bytes: png_bytes})
type(image)

numpy.ndarray